# PicoRV32 Processor Mixed-Memory Processor Demo

This notebook demonstrates using Jupyter Notebooks and IPython Magics to run C/C++ and assembly code on a PicoRV32 Processor. 

The PicoRV32 Processor in this example is a Mixed-Memory processor. This means, it has a 64-KB BRAM Memory space, **AND** a 256 MB DDR Memory space (mapped at address 0x1000_0000). 

When arguments are passed to the PicoRV32 processor for execution they will be copied and passed in **DDR Memory** as PYNQ Contiguous Memory Allocated (CMA) arrays. Previously allocated CMA arrays passed as arguments will not be copied, and reused.

When the program terminates, results are back-propogated (if necessary). 

## Loading the Overlay

To begin, import the overlay using the following cell. This also loads the IPython Magics: `riscvc`, `riscvcpp`, and `riscvasm`. 

In [1]:
from riscvonpynq.picorv32.axi.picorv32 import Overlay
overlay = Overlay("/home/xilinx/jupyter_notebooks/GPCP/designs/picorv32/axi/picorv32.bit")

In [4]:
import pynq

In [10]:
from pynq import PL

In [11]:
print(PL.interrupt_pins)

{'irqConcat/In0': {'controller': 'psInterruptController', 'index': 0, 'fullpath': 'irqConcat/In0'}, 'processor/picorv32/trap': {'controller': 'psInterruptController', 'index': 0, 'fullpath': 'processor/picorv32/trap'}}


In [13]:
pynq.interrupt.Interrupt('irqConcat/In0')

This demonstrates that processor is an instance of riscvonpynq.Processor.MixedProcessor. As we stated above, this means that the processor is connected to BRAM and DDR. riscvonpynq.Processor.MixedProcessor is an indirect subclass of pynq.overlay.DefaultHierarchy -- this means that the processor is actually a collection of IP wrapped in an Block Diagram Editor IP Hierarchy that is recognized by pynq using the `checkhierarchy` method.

The MixedProcessor class provides methods to run, launch (run a program asynchronously), and land (stop an asynchronous program). You can see further documentation in the cell below: 

## RISC-V Magics

Our package provides three RISC-V Magics. The first is `riscvc`, which compiles C code.

In [2]:
%%riscvc test overlay.processor

int main(int argc, char ** argv) {
    unsigned int * a = (unsigned int *)argv[1];
    return a[2];
}

INIT


You can run the test program above and pass it arguments. The arguments must be a Numpy type. 

In [3]:
import numpy as np
arg1 = np.array(range(1, 10), np.uint32)
retval = overlay.processor.run(test, arg1)

if(retval != arg1[2]):
    print("Test Failed!")
else:
    print("Test Passed!")

Test Passed!


The RISC-V Processor lets the ARM Processor know it is complete by raising the IRQ line. Each processor can do this in a different way. For example, the PicoRV32 processor has a `trap` pin that is raised on an `ebreak` instruction. Other processors must write to GPIO pins. 

You can see the IRQ line in the overlay: 

In [6]:
help(overlay.processor.irq)

Help on Interrupt in module pynq.interrupt object:

class Interrupt(builtins.object)
 |  Class that provides the core wait-based API to end users
 |  
 |  Provides a single coroutine wait that waits until the interrupt
 |  signal goes high. If the Overlay is changed or re-downloaded this
 |  object is invalidated and waiting results in undefined behaviour.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, pinname)
 |      Initialise an Interrupt object attached to the specified pin
 |      
 |      Parameters
 |      ----------
 |      pinname : string
 |          Fully qualified name of the pin in the block diagram of the
 |          for ${cell}/${pin}. Raises an exception if the pin cannot
 |          be found in the currently active Overlay
 |  
 |  wait(self)
 |      Wait for the interrupt to be active
 |      
 |      May raise an exception if the Overlay has been changed since
 |      initialisation.
 |  
 |  ------------------------------------------------------------------

You can also examine the processor's memory: 

In [4]:
arr = overlay.processor.psBramController.mmio.array
for i in range(128):  
    print(f'Memory Index {i:3}: {arr[i]:#0{10}x}')

Memory Index   0: 0x00000013
Memory Index   1: 0x00000093
Memory Index   2: 0x00010137
Memory Index   3: 0x00000193
Memory Index   4: 0x00000213
Memory Index   5: 0x00000293
Memory Index   6: 0x00000313
Memory Index   7: 0x00000393
Memory Index   8: 0x00000413
Memory Index   9: 0x00000493
Memory Index  10: 0xffc12503
Memory Index  11: 0xff812583
Memory Index  12: 0x00000613
Memory Index  13: 0x00000693
Memory Index  14: 0x00000713
Memory Index  15: 0x00000793
Memory Index  16: 0x00000813
Memory Index  17: 0x00000893
Memory Index  18: 0x00000913
Memory Index  19: 0x00000993
Memory Index  20: 0x00000a13
Memory Index  21: 0x00000a93
Memory Index  22: 0x00000b13
Memory Index  23: 0x00000b93
Memory Index  24: 0x00000c13
Memory Index  25: 0x00000c93
Memory Index  26: 0x00000d13
Memory Index  27: 0x00000d93
Memory Index  28: 0x00000e13
Memory Index  29: 0x00000e93
Memory Index  30: 0x00000f13
Memory Index  31: 0x00000f93
Memory Index  32: 0x030000ef
Memory Index  33: 0xfeb12c23
Memory Index  

We've also provided Magics for C++ (`riscvcpp`), and Assembly (`riscvasm`). These are demonstrated below: 

In [5]:
%%riscvcpp test_cpp overlay.processor

class foo{
    public:
    static int mulby2(int val){
        return val * 2;
    }
};

int main(int argc, char ** argv){
    int * a = (int *)argv[1];
    return foo::mulby2(a[0]);
}

In [6]:
import numpy as np
test_cpp_arg = np.array([42], np.int32)
retval = overlay.processor.run(test_cpp, test_cpp_arg)

if(retval != test_cpp_arg*2):
    print("Test Failed!")
else:
    print("Test Passed!")

Test Passed!


Finally, some assembly. `int argc` is in register `a0`, and `char **argv` is in register `a1`.

In [7]:
%%riscvasm test_asm overlay.processor

.global main

main:
    lw a2, 4(a1) # Get *argv[1]
    lw a3, 0(a2) # Get argv[1]
    addi a0, a3, -42 # Add -42, store in a0 (return register)
    ret

In [8]:
import numpy as np
test_asm_arg = np.array([42], np.int32)
retval = overlay.processor.run(test_asm, test_asm_arg)

if(retval != test_asm_arg[0] + (-42)):
    print('Test failed!')
else:
    print('Test passed!')

Test passed!


And that's it!